In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)
import time
from sklearn.metrics import accuracy_score
from getData import getData
from ot_dis.mask_map import masking_map, masking_map_sequence, masking_map_partial, masking_map_sequence, masking_map_sequence_2d
from ot_dis.wtk import wtk_distance
from arabic_load import get_train_data, get_test_data

DEBUG:matplotlib:matplotlib data path: c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\Administrator\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\Administrator\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\Administrator\.matplotlib\fontlist-v330.json


In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)
from sklearn.metrics import accuracy_score
from getData import getData
from knn import knn_masking_map, knn_masking_map_pure
from ot_dis.wtk import wtk_distance
from arabic_load import get_train_data, get_test_data

# 1. Load data

In [3]:
X_train, y_train, X_test, y_test = getData('BME', '../data/UCR/raw_data/')
len(X_train[0]), len(X_test)

(128, 150)

In [4]:
X_train_china, y_train_china, X_test_china, y_test_china = getData('Chinatown', '../data/UCR/raw_data/')
len(X_train_china[0]), len(X_train)

(24, 30)

In [5]:
len(X_train), len(X_test), X_train[0].shape

(30, 150, (128,))

In [8]:
X_train_arabic, y_train_arabic = get_train_data()
X_test_arabic, y_test_arabic = get_test_data()
len(X_train_arabic), len(X_test_arabic)

(6600, 2200)

In [7]:
X_train_arabic = X_train_arabic[:660]
X_test_arabic = X_test_arabic[:100]
y_train_arabic = y_train_arabic[:660]
y_test_arabic = y_test_arabic[:100]

In [ ]:
X_train_arabic[0].shape

# 2. Training

## 2.1. Masking map

### 2.1.1. 1D data

In [ ]:
lamb = 5
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="normal",lamb=lamb)
accuracy

### 2.1.2. 2D data

## 2.2. Partial masking map

### 2.2.1. 1D data

In [ ]:
lamb = 5
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="partial",lamb=lamb, s=0.7)
accuracy

### 2.2.2. 2D data

## 2.4. Sequence masking map

In [16]:
import numpy as np
def findTheShortestArray(list1, list2):
    min1 = min(list1, key=len)
    min2 = min(list2, key=len)
    min_len = min(len(min1), len(min2))
    frac1 = int(np.floor(min_len*10/100))
    frac2 = int(np.floor(min_len*30/100))
    frac3 = int(np.floor(min_len*50/100))
    frac4 = int(np.floor(min_len*70/100))
    frac5 = int(np.floor(min_len*90/100))
    min_array = [frac1, frac2, frac3, frac4, frac5]
    return set(min_array)
size_list = [10, 30, 50, 70, 90]

In [32]:
len(X_train_arabic[506])

50

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from ot_dis.mask_map import masking_map_sequence_2d
def knn_masking_map_pure(X_train, X_test, y_train, y_test, method="normal", lamb=5, s=0.5, sub_length=25, k=1):
    
    clf = KNeighborsClassifier(n_neighbors = 1, metric=masking_map_sequence_2d, metric_params={"lamb": lamb, "s": s, "sub_length": sub_length}, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [25]:
accuracy = knn_masking_map_pure(X_train=X_train_arabic, X_test=X_test_arabic, y_train=y_train_arabic, y_test=y_test_arabic,  method="sequence",lamb=3, sub_length=3)
accuracy

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6600,) + inhomogeneous part.

In [23]:
from tqdm import tqdm
train_size = len(X_train_arabic)
test_size = len(X_test_arabic)
result = np.zeros((test_size, train_size))
for train_idx in tqdm(range(train_size)):
    for test_idx in tqdm(range(test_size), leave=False):
        distance = masking_map_sequence_2d(X_train_arabic[train_idx], X_test_arabic[test_idx], lamb=2, sub_length=3)
        result[test_idx, train_idx] = distance

  0%|          | 0/6600 [00:00<?, ?it/s]c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
  0%|          | 3/6600 [01:24<51:48:27, 28.27s/it]


KeyboardInterrupt: 

In [ ]:
sub_length = 40
lamb = 5
accuracy = knn_masking_map_pure(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, sub_length=sub_length)
accuracy